# Goddard Rocket Problem

In [1]:
ENV["TICKTOCK_MESSAGES"] = false;

**Remarques** 
> Il faut l'executer une premiere fois pour pas prendre en compte le temps d'import des librairies.

> MOI.AutomaticDifferentiationBackend() n'est pas supporté par KNITRO

In [2]:
using JuMP
include("../Problems/JuMP/rocket_JMP.jl")
include("./linear_solver_variant.jl")
include("./backend_variant.jl")
include("./solver_variant.jl")
JuMPModel = rocket_JMP();

set_silent(JuMPModel)
set_optimizer(JuMPModel,Ipopt.Optimizer)
set_optimizer_attribute(JuMPModel,"tol",1e-8)
set_optimizer_attribute(JuMPModel,"constr_viol_tol",1e-6)
set_optimizer_attribute(JuMPModel,"max_iter",1000)

## Linear solver

In [ ]:
sl_results,best_ls, plot_ls = linear_solver_variant(JuMPModel);

In [ ]:
if best_ls[1] == "Ipopt : HSLMA27" 
    println("The best linear solver is HSLMA27")
    set_attribute(JuMPModel, "hsllib", HSL_jll.libhsl_path)
    set_attribute(JuMPModel, "linear_solver", "ma27")
elseif best_ls[1] == "Ipopt : HSLMA57"
    println("The best linear solver is HSLMA57")
    set_attribute(JuMPModel, "hsllib", HSL_jll.libhsl_path)
    set_attribute(JuMPModel, "linear_solver", "ma57")
elseif best_ls[1] == "Ipopt : MUMPS"
    println("The best linear solver is MUMPS")
    set_attribute(JuMPModel, "linear_solver", "mumps")
end

## Backend

In [3]:
be_results, best_backend = backend_variant(JuMPModel);

Solving With : JuMPDefault (MathOptInterface.Nonlinear.SparseReverseMode())

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************


Solving With : SymbolicAD (MathOptSymbolicAD.DefaultBackend())

Solving With : ExaModels

Results of Backends :
3×4 DataFrame
 Row │ method       diff_auto_time  total_time  objective_value 
     │ String       Float64         Float64     Float64         
─────┼──────────────────────────────────────────────────────────
   1 │ SymbolicAD        0.0280001   0.0282952          1.01283
   2 │ ExaModels         1.563       2.0659             1.01283
   3 │ JuMPDefault       0.823       2.24173            1.01283


In [4]:
if best_backend == "SymbolicAD" 
    println("The best backend is SymbolicAD")
    set_attribute(JuMPModel,
            MOI.AutomaticDifferentiationBackend(),
            MathOptSymbolicAD.DefaultBackend(),)
elseif best_backend == "JuMPDefault"
    println("The best backend is JuMPDefault")
    set_attribute(JuMPModel,
            MOI.AutomaticDifferentiationBackend(),
            MOI.Nonlinear.SparseReverseMode(),)
elseif best_backend == "ExaModels"
    println("The best linear solver is ExaModels")
    println("ExaModels works only with ipopt and madnlp solvers")
    println("For the sake of the rest of the benchmark, we'll use the 2nd best backend $((be_results[!,1])[2])")
    second_best_backend = (be_results[!,1])[2]
    if second_best_backend == "SymbolicAD" 
        set_attribute(JuMPModel,
                MOI.AutomaticDifferentiationBackend(),
                MathOptSymbolicAD.DefaultBackend(),)
    elseif second_best_backend == "JuMPDefault"
        set_attribute(JuMPModel,
                MOI.AutomaticDifferentiationBackend(),
                MOI.Nonlinear.SparseReverseMode(),)
    end
end

The best backend is SymbolicAD


## solver 

In [5]:
s_results,best_s = solver_variant(JuMPModel);

Solving With Ipopt

Solving With MadNLP

Solving With KNITRO_SQP 

Solving With KNITRO_IPM 

Results of Solvers :
4×4 DataFrame
 Row │ method      diff_auto_time  total_time  objective_value 
     │ String      Float64         Float64     Float64         
─────┼─────────────────────────────────────────────────────────
   1 │ KNITRO_IPM        0.0         0.052718          1.01283
   2 │ KNITRO_SQP        0.140625    2.58228           1.01283
   3 │ Ipopt            15.525      15.5272            1.01283
   4 │ MadNLP            8.504      15.6594            1.01283


In [6]:
if best_s == "KNITRO_IPM" 
    println("The best solver is KNITRO_IPM")
    set_optimizer(JuMPModel,KNITRO.Optimizer);
    set_attribute(JuMPModel, "algorithm", 1);
elseif best_s == "Ipopt"
    println("The best solver is Ipopt")
    set_optimizer(JuMPModel,Ipopt.Optimizer);
elseif best_s == "MadNLP"
    println("The best solver is MadNLP")
    set_optimizer(JuMPModel,MadNLP.Optimizer);
elseif best_s == "KNITRO_SQP"
    println("The best solver is KNITRO_SQP")
    set_optimizer(JuMPModel,KNITRO.Optimizer);
    set_attribute(JuMPModel, "algorithm", 4);
end

The best solver is KNITRO_IPM
